# The Module 3 Project

### Web-Scraping and Data Frame Conversion with Beautiful Soup

Installing necessary libraries

In [2]:
#!conda install -c anaconda lxml 
#!conda install -c anaconda requests
!conda install -c anaconda beautifulsoup4

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    beautifulsoup4: 4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0 anaconda

beautifulsoup4 100% |################################| Time: 0:00:00  37.39 MB/s


My imports:

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

Just using a simple get request to look at the html and obtain appropriate tags (I edited a lot out for the github).

In [7]:
#Now I have a Beautiful Soup object for the table... which I proceed to not really use, but I left it in anyway.
soup = BeautifulSoup(page, 'lxml')
match = soup.find('table', class_ = 'wikitable sortable')
print(match.prettify())

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":876823784,"wgRevisionId":876823784,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","w

In [42]:
#Here is my totally unprocessed pandas Dataframe.
df_toronto = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', skiprows = 1, )[0]
df_toronto.head()

,0,1,2
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [43]:
#Adding column names
df_toronto.rename(columns={ df_toronto.columns[0]: "PostalCode", df_toronto.columns[1]: "Borough", df_toronto.columns[2]: "Neighborhood" }, inplace = True)
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [54]:
#Resetting the index. I only needed the .drop line because I ran the reset an extra time.
df_toronto.query('Borough != "Not assigned"', inplace = True)
df_toronto.reset_index(inplace = True)
df_toronto.drop(['index'], axis=1, inplace = True)
df_toronto.head()


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [55]:
#Checking current shape
print(df_toronto.shape)

(212, 3)


In [93]:
#Joining rows.
df_t = df_toronto.groupby(['PostalCode', 'Borough'], sort = False).agg(lambda x: ', '.join(x))
df_t.head()

,,Neighborhood
PostalCode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront, Regent Park"
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,Not assigned


In [94]:
#Resetting the index
df_t.shape
df_t.reset_index(inplace = True)
df_t.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [96]:
#Fixing the Not assigned line.
df_t.Neighborhood[df_t.Neighborhood == 'Not assigned'] = df_t.Borough[df_t.Neighborhood == 'Not assigned']

In [97]:
#Checking my work.
df_t.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [98]:
#It looks like my joining process took out over a hundred rows.
df_t.shape

(103, 3)

In [ ]:
### 